# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("lr_model").getOrCreate()

In [17]:
df = spark.read.csv('customer_churn.csv',inferSchema=True, header=True)

In [18]:
df.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [9]:
df.show(3)

+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+
|         Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|         Company|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+
| Andrew Mccall|37.0|       9935.53|              1| 7.71|      8.0|2011-08-29 18:37:54|38612 Johnny Stra...|        King Ltd|
|Michele Wright|23.0|       7526.94|              1| 9.28|     15.0|2013-07-22 18:19:54|21083 Nicole Junc...|   Cannon-Benson|
|  Jeremy Chang|65.0|         100.0|              1|  1.0|     15.0|2006-12-11 07:48:13|085 Austin Views ...|Barron-Robertson|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+
only showing top 3 rows



In [10]:
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator, VectorAssembler

In [13]:
location_indexer = StringIndexer(inputCol="Location", outputCol="Location_index")
location_encoder = OneHotEncoderEstimator(inputCols=["Location_index"], outputCols=["Location_encode"])

Company_indexer = StringIndexer(inputCol="Company", outputCol="Company_index")
Company_encoder = OneHotEncoderEstimator(inputCols=["Company_index"], outputCols=["Company_encode"])

In [19]:
assembeler = VectorAssembler(inputCols=["Age", "Total_Purchase", "Account_Manager", 
                                        "Years", "Num_Sites", "Location_encode", "Company_encode"], outputCol="features")

In [14]:
from pyspark.ml.classification import LogisticRegression

In [20]:
model = LogisticRegression(featuresCol="features", labelCol="Churn")

In [21]:
from pyspark.ml import Pipeline

In [46]:
pipeline = Pipeline(stages=[location_indexer,Company_indexer,
                           location_encoder,Company_encoder,
                           assembeler])

In [47]:
df = pipeline.fit(df).transform(df)

In [49]:
train, test = df.randomSplit([0.7, 0.3])

In [50]:
trained_model = model.fit(train)

In [68]:
result = trained_model.transform(test)

In [69]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [81]:
evalutor = BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol="Churn", metricName="areaUnderROC")

In [82]:
evalutor.metricName

Param(parent='BinaryClassificationEvaluator_4fa2a9d877b3db934593', name='metricName', doc='metric name in evaluation (areaUnderROC|areaUnderPR)')

In [83]:
evalutor.evaluate(result)

0.5068376068376068

In [32]:
result.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)
 |-- Location_index: double (nullable = false)
 |-- Company_index: double (nullable = false)
 |-- Location_encode: vector (nullable = true)
 |-- Company_encode: vector (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [84]:
result.show()

+------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+--------------+-------------+-----------------+-----------------+--------------------+--------------------+--------------------+----------+
|             Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|Location_index|Company_index|  Location_encode|   Company_encode|            features|       rawPrediction|         probability|prediction|
+------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+--------------+-------------+-----------------+-----------------+--------------------+--------------------+--------------------+----------+
|       Aaron Meyer|45.0|       9598.03|              0|  5.0|      7.0|2010-07-17 03:30:38|35821 Bailey Skyw...|Steele, Bates and...|    0|         559.0|     